In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [2]:
#df_persona = pd.read_csv('datasets/persona.csv')
df_persona = pd.read_csv("/kaggle/input/persona-csv/persona.csv")
df_persona.head()
df_persona.info()

# unique SOURCE sayısı ve Frekansları
df_persona["SOURCE"].unique()

# unique PRICE
df_persona["PRICE"].nunique()

# Hangi PRICE'dan kaçar tane satış gerçekleştiği bilgisi
df_persona["PRICE"].value_counts()

# Hangi ülkeden kaçar tane satış olduğu bilgisi
df_persona["COUNTRY"].value_counts()

# Soru 6:Ülkelere göre satışlardan kazanılan toplam tutar
df_persona.groupby("COUNTRY")["PRICE"].sum()

# SOURCE türlerine göre satış sayıları
df_persona["SOURCE"].value_counts()

# Ülkelere göre PRICE ortalamaları
df_persona.groupby("COUNTRY")["PRICE"].mean()

# SOURCE'lara göre PRICE ortalamaları
df_persona.groupby("SOURCE")["PRICE"].mean()

# COUNTRY-SOURCE kırılımında PRICE ortalamaları
df_persona.groupby(["COUNTRY", "SOURCE"]).agg({"PRICE": "mean"})


<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

In [3]:
# COUNTRY, SOURCE, SEX, AGE kırılımında ortalama kazançlar
df_persona.groupby(["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}).head()

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897

In [4]:
# Çıktının PRICE’a göre sıralanması
agg_df = df_persona.groupby(by=["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}).sort_values("PRICE", ascending=False)
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


In [5]:
# Indekste yer alan isimleri değişken ismine çevirme
agg_df = agg_df.reset_index()
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


In [6]:
# Age değişkenini kategorik değişkene çevirip yeni DF'e ekleme

bins = [0, 18, 23, 30, 40, agg_df["AGE"].max()]
mylabels = ['0_18', '19_23', '24_30', '31_40', '41_' + str(agg_df["AGE"].max())]

agg_df["AGE_CAT"] = pd.cut(agg_df["AGE"], bins, labels=mylabels)
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,bra,android,male,46,59.0,41_66
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


In [7]:
# Yeni level-based (persona) tanımlama

#### 1

agg_df['customers_level_based'] = agg_df[['COUNTRY', 'SOURCE', 'SEX', 'AGE_CAT']].agg(lambda x: '_'.join(x).upper(), axis=1)
agg_df.head()

#### 2

#agg_df["customers_level_based"] = ['_'.join(i).upper() for i in agg_df.drop(["AGE", "PRICE"], axis=1).values]

##### -age ve price dışındakilere _ uygula diyoruz ###

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based
0,bra,android,male,46,59.0,41_66,BRA_ANDROID_MALE_41_66
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40


In [8]:
# Personaları segmentlere ayırma
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
agg_df.head(30)
agg_df.groupby("SEGMENT").agg({"PRICE": "mean"})

/tmp/ipykernel_17/1351551439.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  agg_df.groupby("SEGMENT").agg({"PRICE": "mean"})


,PRICE
SEGMENT,
D,27.302596
C,32.933339
B,35.436170
A,41.434736


In [9]:
# Yeni gelen müşterilerin sınıflandırılması ve  ne kadar gelir getirebileceği tahmini


#### 33 yaşında IOS kullanan bir Türk kadını hangi segmente aittir ve ortalama ne kadar gelir kazandırması beklenir?

new_user = "TUR_IOS_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]

#### 35 yaşında IOS kullanan bir Fransız kadını hangi segmente ve ortalama ne kadar gelir kazandırması beklenir?

new_user = "FRA_IOS_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based,SEGMENT
208,fra,ios,female,40,33.000000,31_40,FRA_IOS_FEMALE_31_40,C
221,fra,ios,female,31,32.636364,31_40,FRA_IOS_FEMALE_31_40,C
